# Generator of SQL samples
Samples that are generated by this generator could be more sophisticated and more true to SQL, so for example samples that contain multiple nested queries in IN and EXISTS blocks in WHERE statement but it would be just arts for the sake of arts.

This samples have to only show that model can learn on them little more complex structure than just simple mathematic expressions

# Example
SELECT COLUMN_ID, COLUMN_ID, COLUMN_ID FROM TABLE_NAME WHERE COLUMN_ID IN ( STRING, STRING ) OR COLUMN_ID ( NUM, NUM )

WHERE COLUMN_ID FROM TABLE_NAME EXISTS ( subquery )

WHERE COLUMN_ID FROM TABLE_NAME LIKE STRING
WHERE COLUMN_ID FROM TABLE_NAME 
WHERE COLUMN_ID FROM TABLE_NAME LIMIT NUM

In [ ]:
import numpy as np
import random
from sql_simple_samples_generator import *

In [ ]:
print(check_max_length()) # 128, 132, 132, 132

In [ ]:
sample = sql_simple_gen_sample()
print(len(sample))
print(sample)
decoded = sql_simple_decode_sample(sample)
print(decoded)
print(sql_simple_decode_sample_into_text(sample))

In [ ]:
num_of_samples = 32_000
len_of_samples = 140

samples = sql_simple_generate_samples(num_of_samples, len_of_samples)
samples.shape

In [ ]:
data_dir_path = "..\\data\\sql_simple"
for idx, sample in enumerate(samples):
  np.savetxt(f"{data_dir_path}\\sample_no{idx}.txt", sample, newline=" ", fmt="%u")